# Environment setup

In [ ]:
!pip install irc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
import socket
import re
import requests
import time
import os
from datetime import datetime
from threading import Thread, Lock
import pandas as pd
from google.colab import drive

In [ ]:
# mount to Google drive
drive.mount('/content/drive')

# output dir on google drive
OUTPUT_DIR = "/content/drive/MyDrive/twitch_chat_logs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Twitch API settings
CLIENT_ID = "gp762nuuoqcoxypju8c569th9wz7q5"
OAUTH_TOKEN = "oauth:mb053vdgfqc0u2m7folpig78vgxdke"
USERNAME = "danieljia36"
API_URL = "https://api.twitch.tv/helix/streams"
GAMES_API_URL = "https://api.twitch.tv/helix/games"

# Twitch IRC settings
IRC_SERVER = "irc.chat.twitch.tv"
IRC_PORT = 6667

# game list
GAME_LIST = [
    "League of Legends", "Counter-Strike", "Grand Theft Auto V",
    "Assassin's Creed Shadows", "Fortnite", "VALORANT",
    "Marvel Rivals", "Apex Legends", "Rainbow Six Siege"
]

# save mappings between channels and games
joined_channels = {}
sock = None
sock_lock = Lock()

# Store data into Google drive

In [ ]:
def connect():
    global sock
    while True:
        try:
            s = socket.socket()
            s.settimeout(60)
            s.connect((IRC_SERVER, IRC_PORT))
            s.send(f"PASS {OAUTH_TOKEN}\n".encode("utf-8"))
            s.send(f"NICK {USERNAME}\n".encode("utf-8"))
            print("✅ connected to Twitch IRC")
            with sock_lock:
                sock = s
            return s
        except Exception as e:
            print(f"⚠ connection failed，try again in 10s...: {e}")
            time.sleep(10)

def insert_chat_data_batch(rows):
    """ save chat files to Google Drive as csv """
    if not rows:
        return
    df = pd.DataFrame(rows)
    timestamp = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    output_file = os.path.join(OUTPUT_DIR, f"chat_log_{timestamp}.csv")
    df.to_csv(output_file, index=False, encoding="utf-8")
    print(f"✅ save {len(rows)} data {output_file}")

def get_game_ids(game_names):
    headers = {
        "Client-ID": CLIENT_ID,
        "Authorization": f"Bearer {OAUTH_TOKEN.split(':')[1]}"
    }
    game_ids = {}
    for name in game_names:
        response = requests.get(GAMES_API_URL, headers=headers, params={"name": name})
        if response.status_code == 200:
            data = response.json()
            for game in data.get("data", []):
                game_ids[game["name"]] = game["id"]
        else:
            print(f"failed to get game id ({name}): {response.text}")
    return game_ids

def get_live_channels(game_ids):
    headers = {
        "Client-ID": CLIENT_ID,
        "Authorization": f"Bearer {OAUTH_TOKEN.split(':')[1]}"
    }
    live_channels = {}
    for game_name, game_id in game_ids.items():
        params = {"game_id": game_id, "first": 20}
        response = requests.get(API_URL, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            live_channels[game_name] = [stream["user_login"] for stream in data.get("data", [])]
        else:
            print(f"get {game_name} stream list failed: {response.text}")
    return live_channels

def connect_to_channel(s, channel, game_name):
    try:
        s.send(f"JOIN #{channel}\n".encode("utf-8"))
        joined_channels[channel] = game_name
        print(f"joined channel: {channel} (game: {game_name})")
    except Exception as e:
        print(f"join channel {channel} fail: {e}")
        new_sock = connect()
        connect_to_channel(new_sock, channel, game_name)

def listen_chat():
    global sock
    buffer = []
    last_flush = time.time()
    while True:
        try:
            with sock_lock:
                s = sock
            resp = s.recv(2048).decode("utf-8").strip()
            if not resp:
                print("⚠ received empty messages，reconnection...")
                s = connect()
                continue
            if resp.startswith("PING"):
                s.send("PONG :tmi.twitch.tv\n".encode("utf-8"))
                continue
            if "JOIN" in resp:
                print(f"✅ joined channel confirmation: {resp}")
                continue
            if "PRIVMSG" in resp:
                try:
                    user_name = re.search(r":(\w+)!", resp).group(1)
                    message_match = re.search(r"PRIVMSG #([\w]+) :(.*)", resp)
                    channel = message_match.group(1)
                    chat_message = message_match.group(2)
                    timestamp = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
                    game_name = joined_channels.get(channel, "Unknown Game")
                    row = {
                        "timestamp": timestamp,
                        "game_name": game_name,
                        "channel": channel,
                        "user_name": user_name,
                        "message": chat_message
                    }
                    buffer.append(row)
                    if len(buffer) >= 20 or (time.time() - last_flush) >= 5:
                        insert_chat_data_batch(buffer)
                        buffer.clear()
                        last_flush = time.time()
                except Exception as parse_e:
                    print(f"parse messages failed: {parse_e}")
                    continue
        except socket.timeout:
            print("⚠ 60 s no new messages，reconnection...")
            s = connect()
            continue
        except (socket.error, BrokenPipeError) as e:
            print(f"⚠ connection failure: {e}，reconnection...")
            s = connect()
            continue

def update_channels():
    global sock
    game_ids = get_game_ids(GAME_LIST)
    while True:
        live_channels = get_live_channels(game_ids)
        for game_name, channels in live_channels.items():
            for channel in channels:
                with sock_lock:
                    s = sock
                connect_to_channel(s, channel, game_name)
                time.sleep(2)
        time.sleep(60)

In [ ]:
if __name__ == "__main__":
    sock = connect()
    t_listen = Thread(target=listen_chat, daemon=True)
    t_update = Thread(target=update_channels, daemon=True)
    t_listen.start()
    t_update.start()
    t_listen.join()
    t_update.join()

已加入频道: barra (游戏: Tom Clancy's Rainbow Six Siege)
✅ 成功连接到 Twitch IRC
已加入频道: jessgoat (游戏: Tom Clancy's Rainbow Six Siege)
✅ 加入频道确认: :danieljia36!danieljia36@danieljia36.tmi.twitch.tv JOIN #jessgoat
已加入频道: caedrel (游戏: League of Legends)
已加入频道: fojig (游戏: Tom Clancy's Rainbow Six Siege)
✅ 加入频道确认: :danieljia36!danieljia36@danieljia36.tmi.twitch.tv JOIN #fojig
:danieljia36.tmi.twitch.tv 353 danieljia36 = #fojig :danieljia36
:danieljia36.tmi.twitch.tv 366 danieljia36 #fojig :End of /NAMES list
已加入频道: thebausffs (游戏: League of Legends)
已加入频道: jynxzioffline_ (游戏: Tom Clancy's Rainbow Six Siege)
✅ 加入频道确认: :danieljia36!danieljia36@danieljia36.tmi.twitch.tv JOIN #jynxzioffline_
:danieljia36.tmi.twitch.tv 353 danieljia36 = #jynxzioffline_ :danieljia36
:danieljia36.tmi.twitch.tv 366 danieljia36 #jynxzioffline_ :End of /NAMES list
已加入频道: otplol_ (游戏: League of Legends)
已加入频道: oneshooter (游戏: Tom Clancy's Rainbow Six Siege)
已加入频道: jltomy (游戏: League of Legends)
已加入频道: samu_x_funny (游戏: Tom Clancy's

KeyboardInterrupt: 